# RIA - a IA retratista

### A RIA é uma IA especializada em gerar retratos do rosto dos modelos que autorizam a produção. Contudo, a RIA é uma artista incompreendida (nem mesmo ela se compreende) e não salva as artes que produz, por que julga não estarem perfeitas.

### Opções para o retrato:
### - Digite **1** para utilizar lápis de cor
### - Digite **2** para utilizar tinta a base de água
### - Digite **3** para utilizar tinta óleo 

### Depois de ter o autoretrato, digite **c** para outro retrato

### Para encerrar a RIA, digite **q**.

In [ ]:
# Código para baixar os modelos utilizados
# Caso não tenha baixado os modelos, descomente as linhas a seguir e execute

! wget -O face_stylizer_lapis.task -q https://storage.googleapis.com/mediapipe-models/face_stylizer/blaze_face_stylizer/float32/latest/face_stylizer_color_sketch.task
! wget -O face_stylizer_tinta.task -q https://storage.googleapis.com/mediapipe-models/face_stylizer/blaze_face_stylizer/float32/latest/face_stylizer_color_ink.task
! wget -O face_stylizer_oleo.task -q https://storage.googleapis.com/mediapipe-models/face_stylizer/blaze_face_stylizer/float32/latest/face_stylizer_oil_painting.task

In [ ]:
import cv2
import mediapipe as mp

import urllib.request

urls = {
    "face_stylizer_lapis.task": "https://storage.googleapis.com/mediapipe-models/face_stylizer/blaze_face_stylizer/float32/latest/face_stylizer_color_sketch.task",
    "face_stylizer_tinta.task": "https://storage.googleapis.com/mediapipe-models/face_stylizer/blaze_face_stylizer/float32/latest/face_stylizer_color_ink.task",
    "face_stylizer_oleo.task": "https://storage.googleapis.com/mediapipe-models/face_stylizer/blaze_face_stylizer/float32/latest/face_stylizer_oil_painting.task",
}

for name, url in urls.items():
    urllib.request.urlretrieve(url, name)

from mediapipe.tasks import python
from mediapipe.tasks.python import vision
DESIRED_HEIGHT = 680
DESIRED_WIDTH = 680

frame = None
base_options = None
options = None

while True:
    base_options = python.BaseOptions(model_asset_path='face_stylizer_lapis.task')
    cap = cv2.VideoCapture(0) 

    while True:
        encerra = False
        ret, frame = cap.read()
        if not ret:
            print("Falha em obter o frame.")
            break

        cv2.imshow('Camera Feed', frame)

        if cv2.waitKey(10) & 0xFF == ord('1'):
            base_options = python.BaseOptions(model_asset_path='face_stylizer_lapis.task')
            encerra = True
        elif cv2.waitKey(10) & 0xFF == ord('2'):
            base_options = python.BaseOptions(model_asset_path='face_stylizer_tinta.task')
            encerra = True
        elif cv2.waitKey(10) & 0xFF == ord('3'):
            base_options = python.BaseOptions(model_asset_path='face_stylizer_oleo.task')
            encerra = True

        if encerra:
            cap.release()
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            break
    
    options = vision.FaceStylizerOptions(base_options=base_options)
    
    with vision.FaceStylizer.create_from_options(options) as stylizer:
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        stylized_image = stylizer.stylize(mp_image)

        if not stylized_image is None:
            rgb_stylized_image = cv2.cvtColor(stylized_image.numpy_view(), cv2.COLOR_BGR2RGB)
            rgb_stylized_image_resize = cv2.resize(rgb_stylized_image, (DESIRED_WIDTH, DESIRED_WIDTH))
            cv2.imshow('Retrato', rgb_stylized_image_resize)
            if cv2.waitKey(0) & 0xFF == ord('c'):
                cap.release()
                cv2.destroyAllWindows()
                cv2.waitKey(1)

            elif cv2.waitKey(0) & 0xFF == ord('q'):
                break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)